In [1]:
import torch
from torchviz import make_dot

In [31]:
a = torch.tensor([2., 3.], requires_grad=True)
b = torch.tensor([6., 4.], requires_grad=True)

Q = 1/3*a**3 - 1/2*b**2
z = Q.sum()

In [35]:
z_a = torch.autograd.grad(z, a, create_graph=True)[0]
Q_a = torch.autograd.grad(Q.sum(), a, create_graph=True)[0]

In [36]:
print(z)
print(z_a)
print(Q_a)

tensor(-14.3333, grad_fn=<SumBackward0>)
tensor([4., 9.], grad_fn=<MulBackward0>)
tensor([4., 9.], grad_fn=<MulBackward0>)
